In [1]:
import json
import shelve
from tqdm import tqdm
import random
import csv

import matplotlib.pyplot as plt

from data.ag.action_genome import AG, AGViewer
from util.graph_utils import check_edge_exists

VERBS = [
    "awaken",
    "close",
    "cook", 
    "dress",
    "drink",
    "eat",
    "fix",
    "grasp", 
    "hold",
    #"laugh",
    "lie",
    "make",
    "open",
    "photograph",
    "play",
    "pour",
    "put",
    "run",
    "sit",
    #"smile",
    #"sneeze",
    "snuggle",
    "stand",
    "take",
    "talk",
    "throw",
    "tidy",
    "turn",
    "undress",
    "walk",
    "wash",
    "watch",
    "work"
]

root = '/data/Datasets/ag/'
subset_file = 'data/ag/subset_shelve'


In [ ]:

ag = AG(root, no_img=True, split=None, subset_file=subset_file, verb_whitelist=VERBS)
#ag = AG(root, split=None, subset_file=None, verb_blacklist=['smile', 'laugh', 'sneeze']) #view the full dataset
#ag = AG(root, split='test', subset_file='data/ag/subset_shelve') #view the test split dataset
#ag = AG(root, split='train', subset_file='data/ag/subset_shelve') #view the test split dataset


In [ ]:
print(ag.action_verb_obj_map)

In [ ]:
#for a, (v, o) in sorted(ag.action_verb_obj_map.items(), key=lambda x: x[1][0]):
for a, (v, o) in ag.action_verb_obj_map.items():
    action = ag.action_classes[a]
    verb = ag.verb_classes[v]
    obj = None if o is None else ag.object_classes[o]
    print(action, f'({verb}, {obj})')

In [ ]:
%autoreload

with shelve.open('data/ag/subset_shelve') as subset_dict:
    agview = AGViewer(ag, subset_dict)
    agview.view(0)
    while True:
        answer = input(f"{agview.message}current search key : {str(agview.key)}")
        new_idx = agview.process_command(answer)
        if new_idx is not None:
            agview.view(new_idx)
        else:
            break


In [ ]:
%autoreload
idx_list = list(range(len(ag)))
random.shuffle(idx_list)

correct_count = 0
incorrect_count = 0
with shelve.open('data/ag/subset_shelve') as subset_dict:
    agview = AGViewer(ag, subset_dict, human_test=True)
    for idx in idx_list:
        _, verb = agview.view(idx)
        if correct_count + incorrect_count > 0:
            accuracy = correct_count / (correct_count + incorrect_count)
        else:
            accuracy = 0
        answer = input(f"correct: {correct_count}, incorrect: {incorrect_count}, accuracy: {accuracy}")
        if answer == 'q':
            break
        if answer == verb:
            correct_count += 1
            print('CORRECT' + verb)
        else:
            incorrect_count += 1
            print('INCORRECT' + verb)


In [ ]:
#Check for overlap between train, val, and test splits
with open('data/ag/split_train_val_test.json', 'r') as f:
    split_dict = json.load(f)
    train_ids = set(split_dict['train'])
    val_ids = set(split_dict['val'])
    test_ids = set(split_dict['test'])
    
    # Check for overlap
    train_val_overlap = train_ids.intersection(val_ids)
    train_test_overlap = train_ids.intersection(test_ids)
    val_test_overlap = val_ids.intersection(test_ids)

    if len(train_val_overlap) > 0 or len(train_test_overlap) > 0 or len(val_test_overlap) > 0:
        print(f"Warning: Found overlapping IDs between train and val splits")
    else:
        print("No overlap found between train and test splits")


In [ ]:

VERBS = [
    "awaken",
    "close",
    "cook", 
    "dress",
    "drink",
    "eat",
    "fix",
    "grasp", 
    "hold",
    #"laugh",
    "lie",
    "make",
    "open",
    "photograph",
    "play",
    "pour",
    "put",
    "run",
    "sit",
    #"smile",
    #"sneeze",
    "snuggle",
    "stand",
    "take",
    "talk",
    "throw",
    "tidy",
    "turn",
    "undress",
    "walk",
    "wash",
    "watch",
    "work"
]

ag = AG(root, no_img=True, split=None, subset_file=subset_file, verb_whitelist=VERBS)
print(ag.verb_classes)
print(ag.action_classes)
print(ag.action_verb_obj_map)


In [ ]:
freqs = ag.verb_label_counts
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.bar(range(len(freqs)), freqs)
plt.xticks(range(len(freqs)), ag.verb_classes, rotation=45, ha='right')
plt.title('Distribution of Verb Classes')
plt.xlabel('Verb')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


In [ ]:
import pickle

with open(root+'annotations/object_bbox_and_relationship.pkl', 'rb') as f:
    object_annotations = pickle.load(f)

In [14]:
object_annotations['AZC1I.mp4/000453.png']

[{'class': 'laptop',
  'bbox': (230.38460191768993,
   179.69712592394802,
   60.999536911137405,
   29.901400057201265),
  'attention_relationship': ['looking_at'],
  'spatial_relationship': ['in_front_of'],
  'contacting_relationship': ['touching'],
  'metadata': {'tag': 'AZC1I.mp4/laptop/000453', 'set': 'train'},
  'visible': True},
 {'class': 'chair',
  'bbox': (202.32808857808854,
   193.3185564435563,
   59.61538461538461,
   78.46153846153845),
  'attention_relationship': ['not_looking_at'],
  'spatial_relationship': ['beneath', 'behind'],
  'contacting_relationship': ['sitting_on'],
  'metadata': {'tag': 'AZC1I.mp4/chair/000453', 'set': 'train'},
  'visible': True}]

In [16]:
s = 'phone/camera/book'
s.replace('/', '_')

'phone_camera_book'

In [ ]:
import numpy as np
from pyswip import Prolog

ag = AG(root, no_img=True, split='train', subset_file=subset_file, verb_whitelist=VERBS)

def evaluate_rule(verb):
    rules_file = 'outputs/ag/debug_rules.pl'
    bk_file = 'prolog/ag/train_bk.pl'

    _ = Prolog()

    Prolog.consult(rules_file)
    Prolog.consult(bk_file)

    TP, FP, TN, FN = 0, 0, 0, 0

    for i in tqdm(range(len(ag))):
        _, _, _, action, _ = ag[i]

        label_verb, _ = ag.action_verb_obj_map[action]

        q = Prolog.query(f'{verb}_target(x{i}_0)')
        satisfied = False
        for q in q:
            satisfied = True
            break

        if satisfied and label_verb == verb:
            TP += 1
        elif satisfied and label_verb != verb:
            FP += 1
        elif not satisfied and label_verb == verb:
            FN += 1
        else:
            TN += 1

    return TP, FP, TN, FN

evaluate_rule('close')


split: train | length: 4791


   Call: (1) pyrun("consult('outputs/ag/debug_rules.pl')", _6216) ? 

: 